# Purpose
## Author: Nozomi Maki
### Date: 26th March 2023

- I wanna understand how to use libraries used on this notebook


# Updating Python Library Version

In [1]:
%%time

! pip uninstall transformers datasets -y 
! pip install transformers datasets -q

Found existing installation: transformers 4.27.3
Uninstalling transformers-4.27.3:
  Successfully uninstalled transformers-4.27.3
Found existing installation: datasets 2.10.1
Uninstalling datasets-2.10.1:
  Successfully uninstalled datasets-2.10.1
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
CPU times: user 604 ms, sys: 252 ms, total: 855 ms
Wall time: 18.7 s


## Imports

In [2]:
%%time
import datasets
import pandas as pd

import torch
# Because of the Occorence of Error in trying to use tokenizer.decode
"""

File /usr/local/lib/python3.8/site-packages/transformers/utils/generic.py:98, in _is_torch(x)
     97 def _is_torch(x):
---> 98     import torch
    100     return isinstance(x, torch.Tensor)

File /usr/local/lib/python3.8/site-packages/torch/__init__.py:457
    443         raise ImportError(textwrap.dedent('''
    444             Failed to load PyTorch C extensions:
    445                 It appears that PyTorch has loaded the `torch/_C` folder
   (...)
    453                 or by running Python from a different directory.
    454             ''').strip()) from None
    455     raise  # If __file__ is not None the cause is unknown, so just re-raise.
--> 457 for name in dir(_C):
    458     if name[0] != '_' and not name.endswith('Base'):
    459         __all__.append(name)

NameError: name '_C' is not defined

"""

from datasets import load_dataset

CPU times: user 4.21 s, sys: 544 ms, total: 4.75 s
Wall time: 4.66 s


## Dataset

- Official Documentation : [Loading a Dataset](https://huggingface.co/docs/datasets/v1.11.0/loading_datasets.html)

    - [datasets/glue # cola](https://huggingface.co/datasets/glue#cola)
    - According to this article, cola of glue datasets have the feature wheter each document is glamatically correct.

In [3]:
%%time

cola_dataset = load_dataset('glue', 'cola')

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 875 ms, sys: 246 ms, total: 1.12 s
Wall time: 6.62 s


---

Check some versions

In [4]:
! pip show datasets

Name: datasets
Version: 2.10.1
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: aiohttp, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, responses, tqdm, xxhash
Required-by: 


In [5]:
%%time

! pip show transformers 

Name: transformers
Version: 4.27.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, tokenizers, tqdm
Required-by: 
CPU times: user 177 ms, sys: 105 ms, total: 283 ms
Wall time: 5.42 s


---

In [6]:
cola_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [7]:
train_dataset = cola_dataset['train']
val_dataset = cola_dataset['validation']
test_dataset = cola_dataset['test']

In [8]:
len(train_dataset), len(val_dataset), len(test_dataset)

(8551, 1043, 1063)

In [9]:
display(
    train_dataset[0],
    val_dataset[0],
    test_dataset[0]
)

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

{'sentence': 'The sailors rode the breeze clear of the rocks.',
 'label': 1,
 'idx': 0}

{'sentence': 'Bill whistled past the house.', 'label': -1, 'idx': 0}

In [10]:
train_dataset.features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None),
 'idx': Value(dtype='int32', id=None)}

- About filter functions: https://huggingface.co/docs/datasets/v1.1.1/package_reference/main_classes.html#datasets.Dataset.filter


In [11]:
train_dataset.filter(
    lambda example: 
        example['label'] == train_dataset.features['label'].str2int('acceptable')
    )[:5]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-8bed84235e43dc0a.arrow


{'sentence': ["Our friends won't buy this analysis, let alone the next one we propose.",
  "One more pseudo generalization and I'm giving up.",
  "One more pseudo generalization or I'm giving up.",
  'The more we study verbs, the crazier they get.',
  'Day by day the facts are getting murkier.'],
 'label': [1, 1, 1, 1, 1],
 'idx': [0, 1, 2, 3, 4]}

In [12]:
display(
    train_dataset.features['label'].str2int('acceptable'),
    train_dataset.features['label'],
    train_dataset.features,
    train_dataset
)

1

ClassLabel(names=['unacceptable', 'acceptable'], id=None)

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None),
 'idx': Value(dtype='int32', id=None)}

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 8551
})

In [13]:
train_dataset.filter(
    lambda example: 
        example['label'] == train_dataset.features['label'].str2int('unacceptable')
    )[:5]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f819639e03f8949a.arrow


{'sentence': ['They drank the pub.',
  'The professor talked us.',
  'We yelled ourselves.',
  'We yelled Harry hoarse.',
  'Harry coughed himself.'],
 'label': [0, 0, 0, 0, 0],
 'idx': [18, 20, 22, 23, 25]}

## Tokenizing

In [14]:
%%time

from transformers import AutoTokenizer

CPU times: user 248 ms, sys: 53.3 ms, total: 301 ms
Wall time: 332 ms


In [15]:
%%time

tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

CPU times: user 266 ms, sys: 49.7 ms, total: 315 ms
Wall time: 4.4 s


In [16]:
train_dataset = cola_dataset['train']
val_dataset = cola_dataset['validation']
test_dataset = cola_dataset['test']

In [17]:
tokenizer

BertTokenizerFast(name_or_path='google/bert_uncased_L-2_H-128_A-2', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
print(train_dataset[0]['sentence'])
tokenizer(train_dataset[0]['sentence'])

Our friends won't buy this analysis, let alone the next one we propose.


{'input_ids': [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
_temp_sentences = "Our our our fire T our GTGTG T"
print(_temp_sentences)
tokenizer(_temp_sentences)

Our our our fire T our GTGTG T


{'input_ids': [101, 2256, 2256, 2256, 2543, 1056, 2256, 14181, 13512, 2290, 1056, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
%%time

tokenizer.decode(tokenizer(train_dataset[0]['sentence'])['input_ids'])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.56 ms


"[CLS] our friends won't buy this analysis, let alone the next one we propose. [SEP]"

In [21]:
def encode(examples):
    return tokenizer(
            examples["sentence"],
            truncation=True,
            padding="max_length",
            max_length=512,
        )

In [22]:
train_dataset = train_dataset.map(encode, batched=True)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

## Formatting

In [23]:
%%time

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

CPU times: user 3.38 ms, sys: 162 µs, total: 3.55 ms
Wall time: 3.78 ms


## Data Loader

In [24]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)

In [25]:
next(iter(dataloader))

{'label': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
         1, 0, 0, 1, 1, 1, 1, 1]),
 'input_ids': tensor([[  101,  2256,  2814,  ...,     0,     0,     0],
         [  101,  2028,  2062,  ...,     0,     0,     0],
         [  101,  2028,  2062,  ...,     0,     0,     0],
         ...,
         [  101,  5965, 12808,  ...,     0,     0,     0],
         [  101,  2198, 10948,  ...,     0,     0,     0],
         [  101,  3021, 24471,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [26]:
%%time

for batch in dataloader:
    print(batch['input_ids'].shape, batch['attention_mask'].shape, batch['label'].shape)

torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) to

torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])
torch.Size([32, 512]) to